In [ ]:
# Import required libraries
import pandas as pd
from sklearn.svm import SVC
from sklearn.preprocessing import RobustScaler
from sklearn.impute import SimpleImputer
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.ensemble import IsolationForest  # For outlier detection


In [ ]:

# Load the datasets
train_df = pd.read_csv('train_set.csv')
test_df = pd.read_csv('test_set.csv')

# Extract features and target
features = [col for col in train_df.columns if col not in ['ID', 'target']]
X = train_df[features]
y = train_df['target']
test_X = test_df[features]
test_ids = test_df['ID']

# Handle missing values
imputer = SimpleImputer(strategy='mean')
X = pd.DataFrame(imputer.fit_transform(X), columns=features)
test_X = pd.DataFrame(imputer.transform(test_X), columns=features)
contaminations = [0.01, 0.05, 0.1]
# Outlier detection and removal
for elt in contaminations:
  outlier_detector = IsolationForest(contamination=0.05, random_state=42)
  outlier_mask = outlier_detector.fit_predict(X) == 1
  X = X[outlier_mask]
  y = y[outlier_mask]

  # Split the data into training and validation sets
  X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

  # Define a comprehensive parameter grid
  param_grid = {
      'k_best__k': [10, 15, 17, 20, 22],            # More options for feature selection
      'pca__n_components': [0.85, 0.9, 0.925, 0.95, 0.975, 0.99],   # Finer granularity for PCA retained variance
      'model__C': [0.01, 0.1, 0.5, 1, 5, 10, 50, 100, 500, 1000],   # More detailed range of regularization
      'model__gamma': [1e-5, 1e-4, 5e-4, 1e-3, 5e-3, 1e-2, 5e-2, 1e-1, 0.5, 1]  # Finer steps for kernel coefficient
  }


  # Create a pipeline
  pipeline = Pipeline([
      ('scaler', RobustScaler()),        # Scale the features
      ('k_best', SelectKBest(score_func=f_classif)),  # Feature selection
      ('pca', PCA()),                    # Dimensionality reduction
      ('model', SVC(kernel='rbf', random_state=42))  # Non-linear SVM classifier
  ])

  # Perform grid search
  grid_search = GridSearchCV(
      pipeline,
      param_grid,
      cv=5,                              # Cross-validation folds
      scoring='accuracy',
      verbose=2,
      n_jobs=-1                          # Use all CPU cores
  )
  grid_search.fit(X_train, y_train)

# Evaluate on validation set
best_model = grid_search.best_estimator_
y_val_pred = best_model.predict(X_val)
accuracy = accuracy_score(y_val, y_val_pred)
print(f"Validation Accuracy with optimized pipeline: {accuracy}")
print("Best Parameters:", grid_search.best_params_)

# Train the final model on the entire dataset
final_X_selected = X
best_model.fit(final_X_selected, y)
test_predictions = best_model.predict(test_X)

# Save predictions to a CSV file
output_df = pd.DataFrame({'ID': test_ids, 'target': test_predictions})
output_filename = 'non_linear_svm_predictions_extensive_search.csv'
output_df.to_csv(output_filename, index=False)

print(f"Predictions saved to '{output_filename}' with {len(output_df)} rows.")